<a href="https://colab.research.google.com/github/saniya1027108/Zero-Shot-Text-Classification/blob/main/zs_resume_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.0/884.0 KB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 66.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 98.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3f7ba7367d946b90b1208ee17c39e7b1c44cdc0a996f28b93fa63b9699991115
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame(pd.read_csv('/content/test set.csv', encoding = 'latin1'))

In [ ]:
data.rename(columns={'Broad Category':'Broad_Category'}, inplace = True)


In [ ]:
import re
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    resumeText = re.sub("\d+", "", resumeText)  #remove numbers
    resumeText = re.sub("\S*@\S*\s?",' ', resumeText)  #remove emails
    #resumeText = re.sub(r'([^.])( [A-Z]\w*)', r'\1', resumeText) #remove proper nouns, names
    return resumeText

In [ ]:
data['cleaned_resume']= ''
data['cleaned_resume'] = data['resume_string '].apply(cleanResume)

In [ ]:
data['finaltext']= data['cleaned_resume'].values.tolist()
data['finaltext']= data['finaltext'].str.lower()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

     

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data['finaltext'] = data['finaltext'].apply(lambda x: remove_stopwords(x))

In [ ]:
#code for zero-shot-ranking

from transformers import pipeline
import pandas as pd
import ast

In [ ]:
classifier = pipeline("zero-shot-classification", device = 0)

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def zeroshotPipeline(text):
    input_ids = text
    candidate_labels =['AI', 'Web developer', 'big data', 'Enterprise', 'UI UX', 'Cloud developer', 'finance', 'human resource', 'database', 'sales', 'mobile app', 'Customer management', 'admin', 'legal', 'business analyst', 'writing', 'engineering']
    template = "This example is {}"
    results = classifier(input_ids, 
                         candidate_labels,
                        # multi_label = True,
                         hypothesis_template = template)
    score_id = np.argmax(results["scores"])
    final_label = results["labels"][score_id]
    prob = results["scores"][score_id]
    #op = pd.DataFrame(results)
    return final_label ,prob


In [ ]:
data['theme'] = ' '
data['prob'] = np.nan

data['theme'] = data['finaltext'].apply(zeroshotPipeline)


In [ ]:
pd.set_option('display.max_rows', 200)

In [ ]:
data.head(120)

,resume_string,Category,Broad_Category,cleaned_resume,finaltext,theme,prob
0,Name:Kailas P arasram Ra thod \nGender:male \n...,android,Mobile app Developer,Name Kailas P arasram Ra thod Gender male E ma...,name kailas p arasram ra thod gender male e ma...,"(engineering, 0.18007344007492065)",NaN
1,Neelesh Rajendra Atale\nAndroid Developer | Re...,android,Mobile app Developer,Neelesh Rajendra Atale Android Developer React...,neelesh rajendra atale android developer react...,"(mobile app, 0.300104022026062)",NaN
2,Avinash Mali\nAndroid Developer\navi24.9@gmail...,android,Mobile app Developer,Avinash Mali Android Developer avi Mumbai gi...,avinash mali android developer avi mumbai gith...,"(mobile app, 0.2788363993167877)",NaN
3,"SARIT A CHAUDHAR Y\nPhone: +91-9914596184, 987...",android,Mobile app Developer,SARIT A CHAUDHAR Y Phone Email saritachaudh...,sarit chaudhar phone email saritachaudhary sky...,"(mobile app, 0.5231803059577942)",NaN
4,Pg. 1 \n KRISHNA DIPAYAN BHUNIA \n(M. Tech i...,backend dotnet,Web Developer,Pg KRISHNA DIPAYAN BHUNIA M Tech in Software ...,pg krishna dipayan bhunia tech software engg s...,"(admin, 0.14306536316871643)",NaN
5,\nFull Stack .NET Developer with 7+ years of...,backend dotnet,Web Developer,Full Stack NET Developer with years of exper...,full stack net developer years experience eman...,"(Web developer, 0.242000550031662)",NaN
6,ï+919958921027\nï ishukusum28@gmail.com\nï...,bpo,Admin and Customer Support,ishukusum a block goutam apartment kadma Ja...,ishukusum block goutam apartment kadma jamshed...,"(human resource, 0.19923990964889526)",NaN
7,PERRIYIL SOLOMON MATHAI \nPERSONAL DETAIL \n ...,bpo,Admin and Customer Support,PERRIYIL SOLOMON MATHAI PERSONAL DETAIL D O B ...,perriyil solomon mathai personal detail b st m...,"(Enterprise, 0.1723608374595642)",NaN
8,06-08-2020 -\nStill continue\n03-12-2019 -\n...,bpo,Admin and Customer Support,Still continue Shukla Vija yshankar H...,still continue shukla vija yshankar harishanka...,"(engineering, 0.615675151348114)",NaN
9,RESUME \nBhavik V Bhirud \nMobile No: +91 92...,bpo,Admin and Customer Support,RESUME Bhavik V Bhirud Mobile No E Mail bhir...,resume bhavik v bhirud mobile e mail bhirudbha...,"(sales, 0.2436148226261139)",NaN


In [ ]:
split_df = pd.DataFrame(data['theme'].tolist(), columns=['theme', 'score'])

In [ ]:
data.drop('theme')

KeyError: ignored

In [ ]:
data = pd.concat([data, split_df], axis=1)
data.head(10)

,resume_string,Category,Broad_Category,cleaned_resume,finaltext,theme,prob,theme,score,theme,score
0,Name:Kailas P arasram Ra thod \nGender:male \n...,android,Mobile app Developer,Name Kailas P arasram Ra thod Gender male E ma...,name kailas p arasram ra thod gender male e ma...,"(engineering, 0.18007344007492065)",NaN,engineering,0.180073,engineering,0.180073
1,Neelesh Rajendra Atale\nAndroid Developer | Re...,android,Mobile app Developer,Neelesh Rajendra Atale Android Developer React...,neelesh rajendra atale android developer react...,"(mobile app, 0.300104022026062)",NaN,mobile app,0.300104,mobile app,0.300104
2,Avinash Mali\nAndroid Developer\navi24.9@gmail...,android,Mobile app Developer,Avinash Mali Android Developer avi Mumbai gi...,avinash mali android developer avi mumbai gith...,"(mobile app, 0.2788363993167877)",NaN,mobile app,0.278836,mobile app,0.278836
3,"SARIT A CHAUDHAR Y\nPhone: +91-9914596184, 987...",android,Mobile app Developer,SARIT A CHAUDHAR Y Phone Email saritachaudh...,sarit chaudhar phone email saritachaudhary sky...,"(mobile app, 0.5231803059577942)",NaN,mobile app,0.523180,mobile app,0.523180
4,Pg. 1 \n KRISHNA DIPAYAN BHUNIA \n(M. Tech i...,backend dotnet,Web Developer,Pg KRISHNA DIPAYAN BHUNIA M Tech in Software ...,pg krishna dipayan bhunia tech software engg s...,"(admin, 0.14306536316871643)",NaN,admin,0.143065,admin,0.143065
5,\nFull Stack .NET Developer with 7+ years of...,backend dotnet,Web Developer,Full Stack NET Developer with years of exper...,full stack net developer years experience eman...,"(Web developer, 0.242000550031662)",NaN,Web developer,0.242001,Web developer,0.242001
6,ï+919958921027\nï ishukusum28@gmail.com\nï...,bpo,Admin and Customer Support,ishukusum a block goutam apartment kadma Ja...,ishukusum block goutam apartment kadma jamshed...,"(human resource, 0.19923990964889526)",NaN,human resource,0.199240,human resource,0.199240
7,PERRIYIL SOLOMON MATHAI \nPERSONAL DETAIL \n ...,bpo,Admin and Customer Support,PERRIYIL SOLOMON MATHAI PERSONAL DETAIL D O B ...,perriyil solomon mathai personal detail b st m...,"(Enterprise, 0.1723608374595642)",NaN,Enterprise,0.172361,Enterprise,0.172361
8,06-08-2020 -\nStill continue\n03-12-2019 -\n...,bpo,Admin and Customer Support,Still continue Shukla Vija yshankar H...,still continue shukla vija yshankar harishanka...,"(engineering, 0.615675151348114)",NaN,engineering,0.615675,engineering,0.615675
9,RESUME \nBhavik V Bhirud \nMobile No: +91 92...,bpo,Admin and Customer Support,RESUME Bhavik V Bhirud Mobile No E Mail bhir...,resume bhavik v bhirud mobile e mail bhirudbha...,"(sales, 0.2436148226261139)",NaN,sales,0.243615,sales,0.243615


In [ ]:
def get_zero_shot(text):
	sequence = text
	candidate_labels =['artificial intelligence', 'web developer', 'big data', 'Enterprise', 'UI UX', 'Cloud developer', 'finance', 'human resource', 'database', 'sales', 'mobile app', 'Customer management', 'admin', 'legal', 'business analyst', 'writing', 'engineering']
	template = "This example is {}"
  results = classifier(sequence, candidate_labels, hypothesis_template = template)
  score_id = np.argmax(results["scores"])
  final_label = results["labels"][score_id]
  prob = results["scores"][score_id]
  return final_label, prob


data["theme"] = ""
data["prob"] = np.nan

data['theme'] = data["open_text"].apply(zeroshotPipeline)
print(data)
	#output=classifier(sequence, candidate_labels) #input, labels  sequence = entire resume, candidate_labels = all categories (17)
	
	#return output['labels'], output['scores']

IndentationError: ignored

In [ ]:
df_in=pd.read_excel("/content/drive/MyDrive/ss-flexc-folder/4feat-v4331evaluate-index-freq-9jan-IT-subset-testset-13dec.xlsx")
print("\n rows to process = ", df_in.shape[0],"\n columns available = ", df_in.columns)
	


 rows to process =  21 
 columns available =  Index(['Unnamed: 0', 'Unnamed: 0.1', 'S. No. ', '_id', 'user_id',
       'data.Category', 'data.Certification', 'data.SegregatedCertification',
       'data.SkillBlock', 'data.SkillKeywords', 'data.SegregatedSkill',
       'data.Experience', 'data.SegregatedExperience', 'data.JobProfile',
       'segregated_skill_len', 'rec_status', 'golden_skills',
       'validated-gold-skills', 'revisited-gold-skills',
       'resume-semantic-gold-skills', 'jan-validated-god-skills',
       'data.DetailResume', 'word_index', 'word_freq',
       'cl_hp_segregated_skills', 'cl_lp_segregated_skills', 'soft_skills',
       'all_skills', 'skill_feat_col1', 'relevance_occur', 'skill_feat_col2',
       'skill_feat_col3', 'final433', 'show_col', 'stop_col',
       'final_ranked_show', 'normalize_gold', 'gold_len_before',
       'total_skills', 'skills_present'],
      dtype='object')


In [ ]:
all_score=[]
all_rank=[]
for each in df_in.index:
    print("\n at each = ", each)
    cert=df_in['data.Certification'][each]
    skill=df_in['data.SkillBlock'][each]
    exp=df_in['data.Experience'][each]
    combined=""
    if isinstance(cert, str) and len(cert.strip())> 0:
        combined=combined + cert
    if isinstance(skill, str) and len(skill.strip())> 0:
        combined=combined + skill
    if isinstance(exp, str) and len(exp.strip())> 0:
        combined=combined + exp

    labels=ast.literal_eval(df_in['final_ranked_show'][each])
    #print("\n ------ comnined ==",combined, type(labels), labels)
    #y=input("pause 1")

    zs_skill_rank, zs_skill_score= get_zero_shot(combined,labels)

    all_score.append(zs_skill_score)
    all_rank.append(zs_skill_rank)


 at each =  0

 at each =  1

 at each =  2

 at each =  3

 at each =  4

 at each =  5

 at each =  6

 at each =  7

 at each =  8

 at each =  9

 at each =  10

 at each =  11

 at each =  12

 at each =  13

 at each =  14

 at each =  15

 at each =  16

 at each =  17

 at each =  18

 at each =  19

 at each =  20


In [ ]:
#with detailresume section
all_score=[]
all_rank=[]
cl_resume=[]
for each in df_in.index:
    print("\n at each = ", each)
    
    res_text=df_in['data.DetailResume'][each]
    #print("\n before text =", res_text)
    res_text=res_text.replace('\n',' ')
    res_text=res_text.replace('\t',' ')
    res_text=res_text.replace('\r',' ')
    print("\n-----------\n after text = ", res_text)
    cl_resume.append(res_text)


    labels=ast.literal_eval(df_in['final_ranked_show'][each])
    #print("\n ------ comnined ==",combined, type(labels), labels)
    #y=input("pause 1")
    '''
    zs_skill_rank, zs_skill_score= get_zero_shot(res_text,labels)

    all_score.append(zs_skill_score)
    all_rank.append(zs_skill_rank)
 
   '''
df_in['cl_resume']=cl_resume    


 at each =  0

-----------
 after text =  DHIRAJ CHANDRA JHA Mob- 08939588833  Dhirajjha.ab@gmail.com       Experience Summary :  Having 9.8 years of IT experience in Oracle Applications as Techno Functional Finance Consultant of Oracle EBS (11i, R12) .  Extensive experience in implementation, Migration, Customization, Production and support of various modules in Oracle Applications with major focus in Accounts Receivable (AR) , Accounts Payable (AP) ,   General Ledger (GL) , Order Management Experience in Procure to Pay (P2P) , Order to cash (O2C) life cycles. Experience in working with Standard and Custom Reports (RDF & XML) , Forms, Interfaces, Conversions, Basic Workflow Experience in importing data from legacy systems to Oracle Applications using Oracle Interfacing, SQL LOADER, UTL_FILE, Oracle packaged APIs etc. Used WEBADI to upload Vendor Site. Generate XML file using Package and XML Bursting Loaded XML file into Database Table. Invoked REST API using JSON Script via POST Meth

In [ ]:
print(len(all_rank), len(all_score))
df_in['zs_skill_rank']=all_rank
df_in['zs_skill_score']=all_score


21 21


In [ ]:
print("\n rows to process = ", df_in.shape[0],"\n columns available = ", df_in.columns)
	

df_in.to_excel("/content/drive/MyDrive/ss-flexc-folder/cleaned-detail-text-4feat-v4331evaluate-index-freq-9jan-IT-subset-testset-13dec.xlsx")
print("\n done ---------")


 rows to process =  21 
 columns available =  Index(['Unnamed: 0', 'Unnamed: 0.1', 'S. No. ', '_id', 'user_id',
       'data.Category', 'data.Certification', 'data.SegregatedCertification',
       'data.SkillBlock', 'data.SkillKeywords', 'data.SegregatedSkill',
       'data.Experience', 'data.SegregatedExperience', 'data.JobProfile',
       'segregated_skill_len', 'rec_status', 'golden_skills',
       'validated-gold-skills', 'revisited-gold-skills',
       'resume-semantic-gold-skills', 'jan-validated-god-skills',
       'data.DetailResume', 'word_index', 'word_freq',
       'cl_hp_segregated_skills', 'cl_lp_segregated_skills', 'soft_skills',
       'all_skills', 'skill_feat_col1', 'relevance_occur', 'skill_feat_col2',
       'skill_feat_col3', 'final433', 'show_col', 'stop_col',
       'final_ranked_show', 'normalize_gold', 'gold_len_before',
       'total_skills', 'skills_present', 'cl_resume'],
      dtype='object')

 done ---------
